# Data Generation

Notebook to generate four different simulated datasets: Randomized, Plane Wave, Ring Wave, and Rotating Wave.

## Imports

In [1]:
# Standard library imports
import random
import pickle

# Data analysis packages
import numpy as np

# Braingeneers packages for analysis
import braingeneers
from braingeneers.analysis.analysis import SpikeData

## Helper Functions

In [2]:
def pickle_object(obj, filename):
    """
    Serializes an object and saves it to a file using the pickle protocol.

    Parameters:
    - obj: The Python object to be pickled.
    - filename: The name of the file where the object will be stored.
    """
    with open(filename, "wb") as file:
        pickle.dump(obj, file)

## Random Data Code

In [3]:
def generate_randomized_spike_data(
    grid_size=16,
    recording_duration_ms=60000,
    mean_firing_rate_hz=3.45,
    firing_rate_std_dev_hz=0.9,
):
    """
    Generates spike times for neurons arranged in a grid_size x grid_size grid, simulating
    a high-density multi-electrode array (HD MEA). Each neuron's firing rate is randomly
    determined by a normal distribution centered around mean_firing_rate_hz with a
    standard deviation of firing_rate_std_dev_hz, reflecting the lack of inherent spatial
    or temporal patterns.

    Parameters:
    - grid_size: The width and height of the square neuron grid, representing the HD MEA.
    - recording_duration_ms: The duration of the simulated recording in milliseconds.
    - mean_firing_rate_hz: The mean firing rate in Hertz (Hz) for the neuron firing rate distribution.
    - firing_rate_std_dev_hz: The standard deviation in Hz of the neuron firing rates.

    Returns:
    - A SpikeData object containing:
        - spike_times_list: A list of numpy arrays, each array containing the spike times for each neuron.
        - total_neurons: The total number of neurons simulated.
        - recording_length_ms: The total duration of the recording in milliseconds.
        - neuron_layout: A dictionary mapping each neuron to its position on the grid.
    """
    np.random.seed(0)  # For reproducibility
    total_neurons = grid_size * grid_size
    spike_times_list = []

    # Generate spike times with variable firing rates for each neuron
    for _ in range(total_neurons):
        # Determine the neuron's firing rate from a normal distribution
        neuron_firing_rate_hz = np.random.normal(
            mean_firing_rate_hz, firing_rate_std_dev_hz
        )
        neuron_firing_rate_hz = max(
            neuron_firing_rate_hz, 0
        )  # Ensure the rate is not negative
        num_spikes = np.random.poisson(
            neuron_firing_rate_hz * recording_duration_ms / 1000
        )  # Convert Hz to spikes per ms
        spike_times = np.random.uniform(0, recording_duration_ms, num_spikes)
        spike_times_list.append(spike_times)

    # Generate positions for each neuron on the grid
    neuron_positions = [(x, y) for x in range(grid_size) for y in range(grid_size)]
    neuron_layout = {
        "positions": {
            i: {"position": neuron_positions[i]} for i in range(total_neurons)
        }
    }

    # Package the generated spike data and metadata into a SpikeData object
    randomized_spike_data = SpikeData(
        spike_times_list,
        N=total_neurons,
        length=recording_duration_ms,
        neuron_data=neuron_layout,
    )
    return randomized_spike_data

## Plane Wave Data Generator

In [4]:
def generate_wave_propagation_spike_data(
    total_duration_ms,
    num_waves,
    num_rows,
    num_cols,
    base_overlap_duration_ms,
    inter_wave_break_ms,
    sparsity_factor=20,
    wave_slant_delay_ms=1,
    overlap_duration_variation_ms=5,
):
    """
    Simulates coordinated neuronal activity reminiscent of plane waves in the neocortex,
    generating a structured dataset that models a sequence of neuronal spikes across
    a multi-electrode array (MEA). This function aims to emulate a plane-wave-like
    pattern of activity, inspired by observations in Wu's "Propagating Waves of
    Activity in the Neocortex" paper.

    Parameters:
    - total_duration_ms: Total duration of the simulation in milliseconds.
    - num_waves: Number of distinct propagation waves to generate.
    - num_rows, num_cols: Dimensions of the MEA grid.
    - base_overlap_duration_ms: Baseline overlap duration in milliseconds for adjacent rows.
    - inter_wave_break_ms: Duration of the break period in milliseconds between waves.
    - sparsity_factor: Controls the sparsity of spikes; higher values mean fewer spikes.
    - wave_slant_delay_ms: Delay in ms as the wave moves from one column to the next.
    - overlap_duration_variation_ms: Maximum variation in milliseconds from the baseline overlap duration.

    Returns:
    - A SpikeData object containing:
      - `train`: A list of numpy arrays, each array contains spike times for a neuron.
      - `N`: Total number of channels (neurons).
      - `length`: Total duration of the simulated spike data in milliseconds.
      - `neuron_data`: Dictionary with neuron positions in the MEA grid.
    """

    # Initialize variables and structures
    total_channels = num_rows * num_cols
    effective_simulation_duration_ms = total_duration_ms - (
        inter_wave_break_ms * (num_waves - 1)
    )
    wave_duration_ms = effective_simulation_duration_ms / num_waves
    neuron_positions = [(x, y) for x in range(num_rows) for y in range(num_cols)]
    neuron_positions_data = {
        "positions": {
            i: {"position": neuron_positions[i]} for i in range(total_channels)
        }
    }
    spike_times_by_channel = [[] for _ in range(total_channels)]

    # Generate spikes for each propagation wave
    for wave in range(num_waves):
        wave_start = wave * (wave_duration_ms + inter_wave_break_ms)
        current_overlap_duration_ms = base_overlap_duration_ms + np.random.uniform(
            -overlap_duration_variation_ms, overlap_duration_variation_ms
        )

        for row in range(num_rows):
            row_start = wave_start + (
                row * (wave_duration_ms / num_rows - current_overlap_duration_ms)
            )

            for col in range(num_cols):
                channel_index = row * num_cols + col
                spike_time_start = row_start + (col * wave_slant_delay_ms)
                spike_time_end = (
                    spike_time_start
                    + (wave_duration_ms / num_rows)
                    + current_overlap_duration_ms
                )
                num_spikes = int(wave_duration_ms / num_rows / sparsity_factor)
                random_spike_times = np.random.uniform(
                    spike_time_start, spike_time_end, num_spikes
                )
                random_spike_times.sort()

                spike_times_by_channel[channel_index].extend(random_spike_times)

    # Convert spike times to numpy arrays and package in SpikeData object
    spike_times_by_channel = [np.array(times) for times in spike_times_by_channel]
    plane_wave_spike_data = SpikeData(
        train=spike_times_by_channel,
        N=total_channels,
        length=total_duration_ms,
        neuron_data=neuron_positions_data,
    )

    return plane_wave_spike_data

## Ring Wave Data Generator

In [5]:
def generate_ring_wave_spike_data(
    total_duration_ms,
    num_waves,
    grid_rows,
    grid_cols,
    base_overlap_duration_ms,
    break_duration_ms,
    overlap_variation_ms=5,
):
    """
    Simulates repeated circular propagations of neural activity, resembling ripples created by dropping
    a stone into water. This pattern is inspired by "ring wave" patterns observed in rodent hippocampal slices.
    The simulation creates waves that begin with a small ring of neurons at the center of the MEA, firing
    simultaneously and propagating outward. The firing times of subsequent rings are determined by their
    distance from the center, creating a ripple effect.

    Parameters:
    - total_duration_ms: Total duration of the simulation in milliseconds.
    - num_waves: Number of circular waves to generate.
    - grid_rows, grid_cols: Dimensions of the MEA grid.
    - base_overlap_duration_ms: Baseline overlap duration in milliseconds for adjacent neuron rings.
    - break_duration_ms: Duration of the break period in milliseconds between waves.
    - overlap_variation_ms: Maximum variation in milliseconds from the baseline overlap duration.

    Returns:
    - A SpikeData object containing:
        - spike_times_list: A list of numpy arrays, each array containing the spike times for each neuron.
        - total_neurons: The total number of neurons simulated.
        - recording_length_ms: The total duration of the recording in milliseconds.
        - neuron_layout: A dictionary mapping each neuron to its position on the grid.
    """
    total_neurons = grid_rows * grid_cols
    adjusted_total_duration_ms = total_duration_ms - (
        break_duration_ms * (num_waves - 1)
    )
    wave_duration_ms = adjusted_total_duration_ms / num_waves

    center_x, center_y = (grid_rows - 1) / 2, (grid_cols - 1) / 2
    max_distance = np.sqrt(center_x**2 + center_y**2)

    neuron_positions = [(x, y) for x in range(grid_rows) for y in range(grid_cols)]
    neuron_layout = {
        "positions": {
            i: {"position": neuron_positions[i]} for i in range(total_neurons)
        }
    }
    distances = [
        np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2) for x, y in neuron_positions
    ]

    spike_times_by_neuron = [[] for _ in range(total_neurons)]

    preassigned_firings = {
        neuron_index: random.randint(0, num_waves - 1)
        for neuron_index in range(total_neurons)
    }

    for wave in range(num_waves):
        wave_start = wave * (wave_duration_ms + break_duration_ms)
        current_overlap_duration_ms = base_overlap_duration_ms + np.random.uniform(
            -overlap_variation_ms, overlap_variation_ms
        )

        for neuron_index, distance in enumerate(distances):
            firing_time_within_wave = (distance / max_distance) * (
                wave_duration_ms - current_overlap_duration_ms
            )
            spike_time_start = wave_start + firing_time_within_wave
            spike_time_end = spike_time_start + current_overlap_duration_ms

            # Generate random spike times within the overlap duration
            num_spikes = int(current_overlap_duration_ms / 10)
            random_spike_times = np.random.uniform(
                spike_time_start, spike_time_end, num_spikes
            )
            random_spike_times.sort()

            spike_times_by_neuron[neuron_index].extend(random_spike_times)

    spike_times_by_neuron = [np.array(times) for times in spike_times_by_neuron]
    ring_wave_spike_data = SpikeData(
        spike_times_by_neuron,
        N=total_neurons,
        length=total_duration_ms,
        neuron_data=neuron_layout,
    )

    return ring_wave_spike_data

## Rotating Loop Wave Data Generator

In [6]:
def generate_rotating_circle_wave_data(
    total_duration_ms,
    num_rotations,
    grid_rows,
    grid_cols,
    base_overlap_duration_ms,
    overlap_variation_ms=5,
):
    """
    Generate spike data for simulating a rotating circle of firing neurons.

    Args:
    - total_duration_ms (int): Total duration of the simulation in milliseconds.
    - num_rotations (int): Number of rotations for the circle.
    - grid_rows (int): Number of rows in the grid representing neurons.
    - grid_cols (int): Number of columns in the grid representing neurons.
    - base_overlap_duration_ms (int): Base duration of overlap between neuron firings.
    - overlap_variation_ms (int, optional): Variability in overlap duration in milliseconds.

    Returns:
    - rotating_circle_wave_spike_data (SpikeData): Spike data for the rotating circle wave simulation.
    """

    total_neurons = grid_rows * grid_cols
    rotation_duration_ms = total_duration_ms / num_rotations

    # Calculate parameters for the rotating circle
    center_x, center_y = (grid_rows - 1) / 2, (grid_cols - 1) / 2
    max_distance = np.sqrt(center_x**2 + center_y**2)
    circle_radius = max_distance / 2  # Circle at half the max distance
    thickness = 2  # Thickness of the rotating circle in neurons

    # Generate positions, distances, and angles for all neurons
    neuron_positions = [(x, y) for x in range(grid_rows) for y in range(grid_cols)]
    distances = [
        np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2) for x, y in neuron_positions
    ]
    angles = [np.arctan2(y - center_y, x - center_x) for x, y in neuron_positions]

    # Select neurons within the rotating circle
    rotating_circle_neurons = [
        (index, pos, dist, angle)
        for index, (pos, dist, angle) in enumerate(
            zip(neuron_positions, distances, angles)
        )
        if circle_radius - thickness <= dist <= circle_radius + thickness
    ]
    rotating_circle_neurons_sorted = sorted(
        rotating_circle_neurons, key=lambda x: x[3]
    )  # Sort by angle

    # Initialize spike times for each neuron
    spike_times_by_neuron = [[] for _ in range(total_neurons)]

    for rotation in range(num_rotations):
        rotation_start = rotation * rotation_duration_ms
        current_overlap_duration_ms = base_overlap_duration_ms + np.random.uniform(
            -overlap_variation_ms, overlap_variation_ms
        )

        for i, (neuron_index, _, _, _) in enumerate(rotating_circle_neurons_sorted):
            position_in_rotation = i / len(rotating_circle_neurons_sorted)
            firing_time_within_rotation = position_in_rotation * (
                rotation_duration_ms - current_overlap_duration_ms
            )
            spike_time_start = rotation_start + firing_time_within_rotation
            spike_time_end = spike_time_start + current_overlap_duration_ms

            # Generate random spike times within the overlap duration
            num_spikes = int(current_overlap_duration_ms / 10)
            random_spike_times = np.random.uniform(
                spike_time_start, spike_time_end, num_spikes
            )
            random_spike_times.sort()  # Ensures chronological order

            spike_times_by_neuron[neuron_index].extend(random_spike_times)

    # Convert spike times to arrays and create neuron layout dictionary
    spike_times_by_neuron = [np.array(times) for times in spike_times_by_neuron]
    neuron_layout = {
        "positions": {
            i: {"position": neuron_positions[i]} for i in range(total_neurons)
        }
    }

    # Create SpikeData object for the rotating circle wave simulation
    rotating_circle_wave_spike_data = SpikeData(
        spike_times_by_neuron,
        N=total_neurons,
        length=total_duration_ms,
        neuron_data=neuron_layout,
    )
    return rotating_circle_wave_spike_data

## Generation

In [7]:
random_spike_data = generate_randomized_spike_data(
    grid_size=16,
    recording_duration_ms=60000,
    mean_firing_rate_hz=3.45,
    firing_rate_std_dev_hz=0.9,
)

plane_wave_spike_data = generate_wave_propagation_spike_data(
    total_duration_ms=60000,
    num_waves=10,
    num_rows=16,
    num_cols=16,
    base_overlap_duration_ms=100,
    inter_wave_break_ms=2000,
    sparsity_factor=20,
    wave_slant_delay_ms=10,
    overlap_duration_variation_ms=5,
)

ring_wave_spike_data = generate_ring_wave_spike_data(
    total_duration_ms=60000,
    num_waves=10,
    grid_rows=16,
    grid_cols=16,
    base_overlap_duration_ms=200,
    break_duration_ms=2000,
    overlap_variation_ms=10,
)

rotating_circle_spike_data = generate_rotating_circle_wave_data(
    total_duration_ms=60000,
    num_rotations=10,
    grid_rows=16,
    grid_cols=16,
    base_overlap_duration_ms=100,
    overlap_variation_ms=5,
)

## Save datasets

In [8]:
pickle_object(random_spike_data, "data/random_spike_data.pkl")
pickle_object(plane_wave_spike_data, "data/plane_wave_spike_data.pkl")
pickle_object(ring_wave_spike_data, "data/ring_wave_spike_data.pkl")
pickle_object(rotating_circle_spike_data, "data/rotating_circle_spike_data.pkl")